In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import signal
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, GroupKFold
from sklearn.neural_network import MLPClassifier


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
EEG_SAMPLING_RATE = 200
EEG_LENGTH = 50
AMOUNT_OF_CHANNELS = 20

In [3]:
data = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
# tensor_data = torch.from_numpy(data) #torch.tensor(data.values, dtype=torch.float32)

In [4]:
TARGETS = ["seizure_vote", "lpd_vote", "gpd_vote", "lrda_vote", "grda_vote", "other_vote"]
y_data = data[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
data[TARGETS] = y_data

data.head()

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,1.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# https://www.kaggle.com/code/kimbyungchun/preprocess-with-mne-for-human
# @cache
BASE_PATH = '/kaggle/input/hms-harmful-brain-activity-classification'
def load_eeg(eeg_id, data_type='train'):
    eeg_df = pd.read_parquet(f'{BASE_PATH}/{data_type}_eegs/{eeg_id}.parquet')
    return eeg_df

In [6]:
# def extract_eeg_features(df, sampling_rate):
#   """
#   Extracts features from an EEG signal in a pandas DataFrame.

#   Args:
#       df: A pandas DataFrame with shape (times, channels) containing the EEG data.
#       sampling_rate: The sampling rate of the EEG data in Hz.

#   Returns:
#       A pandas DataFrame containing the extracted features.
#   """

#   features_df = pd.DataFrame()

#   # Time-domain features
#   features_df["mean"] = df.mean(axis=0)
#   features_df["std"] = df.std(axis=0)
#   features_df["variance"] = df.var(axis=0)
#   features_df["max"] = df.max(axis=0)
#   features_df["min"] = df.min(axis=0)

#   # Frequency-domain features
#   eeg_data_array = df.to_numpy()  # Convert DataFrame to NumPy array
#   fft_data = np.fft.fft(eeg_data_array, axis=0)  # Perform FFT
#   power_spectrum = np.abs(fft_data) ** 2
#   print(fft_data.shape)
#   print(type(fft_data))

#   for channel in range(df.shape[1]):
#     features_df[f"power_delta_{channel}"] = np.sum(power_spectrum[:, 1:4])  # Delta band (1-4 Hz)
#     features_df[f"power_theta_{channel}"] = np.sum(power_spectrum[:, 4:8])  # Theta band (4-8 Hz)
#     features_df[f"power_alpha_{channel}"] = np.sum(power_spectrum[:, 8:13])  # Alpha band (8-13 Hz)
#     features_df[f"power_beta_{channel}"] = np.sum(power_spectrum[:, 13:30])  # Beta band (13-30 Hz)

#   return features_df

In [7]:
# row = data.iloc[0]
# eeg = processRow(row)
# print("d")

# print(eeg.shape)
# print(type(eeg))
# display(eeg.tail())

In [8]:
# # display(train.head())
# labels = data['expert_consensus']

# encoder = LabelEncoder()
# encoded_labels = encoder.fit_transform(labels)

# data['expert_consensus_enc'] = encoded_labels



In [9]:
eeg_dict = {}
def getEEG(row, data_type):
#     print("Loading EEG row:", row.name)
    if not row.eeg_id in eeg_dict:
        df = load_eeg(row.eeg_id, data_type)  
        no_nans = df.dropna()
        eeg_dict[row.eeg_id] = no_nans
    eeg = eeg_dict[row.eeg_id]
    
    start_eeg = int(row.eeg_label_offset_seconds * EEG_SAMPLING_RATE)
    end_eeg = start_eeg + (EEG_LENGTH * EEG_SAMPLING_RATE)

    row_eeg = eeg[start_eeg:end_eeg]

#     features = extract_eeg_features(row_eeg, EEG_SAMPLING_RATE)
    return row_eeg


In [10]:
# from sklearn.base import BaseEstimator, TransformerMixin
# class CustomTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self):
# #         self.parameter1 = parameter1
# #         self.parameter2 = parameter2
#         pass
#     def fit(self, X, y=None):
#         # Calculate statistics or learn parameters from X if needed
#         return self

#     def transform(self, X):
#         # Apply the transformation to X
#         print("Total transforms:", X.shape)
#         X = X.apply(processRow, axis=1)
#         print("transformer done")
        
      
#         return X

In [11]:
# TIME_BINS_PER_SECOND = 10
# BIN_LENGTH = int((1/TIME_BINS_PER_SECOND)*EEG_SAMPLING_RATE)
# def extract_time_bins(row_eeg, key):
#     middle_10 = row_eeg.iloc[20*EEG_SAMPLING_RATE:30*EEG_SAMPLING_RATE]
#     new_fts = pd.DataFrame()
#     for chan in range(middle_10.shape[1]):
#         col_eeg = middle_10.iloc[:, chan]
#         col_eeg = col_eeg.reset_index(drop=True)
#         plt.plot(col_eeg)
#         bin_mns = []
#         for bin_index in range(100):
#             start_index = bin_index * BIN_LENGTH
#             end_index = start_index + BIN_LENGTH
# #             print(start_index)
# #             print(end_index)
            
#             bin_eeg = col_eeg[start_index:end_index]

# #             print(col_eeg.loc[start_index])
#             mn = bin_eeg.mean()
#             bin_mns.append(mn)
# #             print(mn)
# #         print(bin_mns)
# #         break
#         new_fts["ch"+str(chan)] = bin_mns
# #         print(new_fts)
#         break

#     return new_fts
            
# #         print(type(col_eeg))
# #         print(col_eeg)
# #         break

In [12]:
# def find_eeg_id(eeg_id):
#     row = data.loc[data['eeg_id'] == eeg_id].head(1)
#     return row

    
# find_eeg_id(1969473879)

In [13]:
# row = data.iloc[10827]
# display(row)
# row_eeg = getEEG(data.iloc[0], "train")
# key = str(row.eeg_id) + "-" + str(row.eeg_sub_id)
# new_fts = extract_time_bins(row_eeg, key)
# display(new_fts)

In [14]:
from scipy import signal
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
import torch
import torch.utils.data as data_utils
import math

def extract_features(row_eeg, key):
#     print(row_eeg)
    data = []
    for chan in range(row_eeg.shape[1]):
        col_eeg = row_eeg.iloc[:, chan]
        
        # display(row_eeg.iloc[:, 0])
        frequencies, psd = signal.welch(col_eeg, EEG_SAMPLING_RATE, nperseg=512)
        # Select frequencies below 20 Hz
        indices_below_20Hz = frequencies < 20
        selected_frequencies = frequencies[indices_below_20Hz]
        selected_psd = psd[indices_below_20Hz]


        # print(col_eeg)
        mean_value = np.mean(col_eeg)
        variance_value = np.var(col_eeg)
        skewness_value = float(skew(col_eeg))
        if math.isnan(skewness_value):
            skewness_value = 0
        kurtosis_value = kurtosis(col_eeg)
        if math.isnan(float(kurtosis_value)):
            kurtosis_value = 0
        
        tmp = [mean_value, variance_value, skewness_value, kurtosis_value]
        tmp.extend(selected_psd)
        if (np.isnan(tmp).any()):
#             freq_plot(selected_frequencies, selected_psd)
            print("WARNING NAN", key)
            print("isnan4", [mean_value, variance_value, skewness_value, kurtosis_value])
            print(selected_frequencies)
            print(selected_psd)
            print(col_eeg)
            print("eeg has nan", np.count_nonzero(np.isnan(data)))
#             return False
#         print(selected_psd.shape)
#         tmp = torch.tensor(tmp, dtype=torch.float32)
        
        data.append(tmp)
#     print(data)
    return data
#                                     pd.DataFrame(data)

In [15]:
# Plot the results
def freq_plot(selected_frequencies, selected_psd):
    plt.figure(figsize=(8, 4))
    plt.semilogy(selected_frequencies, selected_psd)
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('PSD [V**2/Hz]')
    plt.title('Power Spectral Density (Below 20 Hz)')
    plt.grid(True)
    plt.show()
#     print(selected_frequencies)

In [16]:
features_dict = {}

In [17]:
def getAndExtractEEG(row, data_type):
    key = str(row.eeg_id) + "-" + str(row.eeg_sub_id)
    if not key in features_dict:
        print("New", end = " ")
        row_eeg = getEEG(row, data_type)
        features = extract_features(row_eeg, key)
#         if not features:
#             return
        features_dict[key] = features
        return features
    print("mem", end=" ")
    return features_dict[key]
#     eeg = eeg_dict[row.eeg_id]
    


In [18]:
# df = eeg_dict[3630961636]
# df[df.isna().any(axis=1)]
# # df.iloc[4802]

In [19]:
# df = data.loc[data['eeg_id'] == 3630961636]
# print(df.iloc[)
# getAndExtractEEG(df.iloc[0]) 
# 3630961636-0

In [20]:
# This piece of code is required to make use of the GPU instead of CPU for faster processing
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [21]:
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [22]:

# torch.autograd.set_detect_anomaly(True)

In [23]:
# import pickle
# with open('my_dict.pkl', 'rb') as file:
#     features_dict = pickle.load(file)

In [24]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
#         self.batch_norm = nn.BatchNorm1d(hidden_dim)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        self.batch_norm2 = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(p=0.5)  # Add dropout layer
        self.fc4 = nn.Linear(hidden_dim, 600)
        self.sigmoid = nn.Sigmoid()
        self.fc3 = nn.Linear(600, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.batch_norm2(x)
        x = self.dropout(x)  # Apply dropout

        x = self.fc4(x)
        x = self.sigmoid(x)
        x = self.fc3(x)
        output = self.softmax(x)
        return output

# Example usage
input_dim = (52 + 4) * AMOUNT_OF_CHANNELS
hidden_dim = 800
output_dim = 6


In [25]:
# from sklearn.pipeline import Pipeline
import torch.nn.functional as F


kf = KFold(n_splits=20, shuffle=True, random_state=42)
    
  # Store scores from each fold
cv_scores = []

cv_data = data.iloc[:10000]

# cv_data = data.sample(10000)
# print(type(cv_data))
# print(cv_data)

# print(data)
  # Iterate through each fold
    
batch_size = 32
num_epochs = 11
for train_index, test_index in kf.split(cv_data):
    # Split data into training and testing sets
    X_train, X_test = cv_data.iloc[train_index], cv_data.iloc[test_index]
    y_train, y_test = cv_data.iloc[train_index][TARGETS], cv_data.iloc[test_index][TARGETS]
   
#     X_train = X_train.apply(getAndExtractEEG, axis=1)
    X_train_proc = []
    for index, row in X_train.iterrows():
        X_train_proc.append(getAndExtractEEG(row, "train"))
    X_train_proc = torch.FloatTensor(X_train_proc)
    
    X_test_proc = []
    for index, row in X_test.iterrows():
        X_test_proc.append(getAndExtractEEG(row, "train"))
    X_test_proc = torch.FloatTensor(X_test_proc)
    X_test_proc= X_test_proc.view(-1, input_dim)
        
#     X_test = X_test.apply(getAndExtractEEG, axis=1)
    
    y_train = torch.tensor(np.asarray(y_train), dtype=torch.float32)
    y_test = torch.tensor(np.asarray(y_test), dtype=torch.float32)
    
    train_dataset = MyDataset(X_train_proc, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    model = MLP(input_dim, hidden_dim, output_dim)
    # Define loss function and optimizer (replace with your choices if needed)
    criterion = nn.KLDivLoss(reduction="batchmean")
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    seen_ys = [0,0,0,0,0,0]
    
    for epoch in range(num_epochs):
        
        for batch_X, batch_y in train_loader:
#             print(batch_X.shape)
            batch_X= batch_X.view(-1, input_dim)
            batch_X = batch_X.to(device)
        
            seen_ys = seen_ys+ torch.sum(batch_y, dim=0).numpy()
            
            batch_y = batch_y.to(device)
            
            

            optimizer.zero_grad()

            # Your model forward pass here
            prediction = model(batch_X)
            
            # Calculate loss
            loss = criterion(prediction, batch_y)
            # Compute gradients
            loss.backward()
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=args.clip)
            # Update weights
            optimizer.step()
#             print("One step done")
            
    # Evaluate the model on the validation set
    with torch.no_grad():
        model.eval()
        print(X_test_proc)
        pred_probs = model(X_test_proc)
        print(pred_probs)
        print(y_test)
#         kl_divergence = F.kl_div(pred_probs.log(), y_test, reduction='batchmean')
        loss = criterion(pred_probs, y_test)
        print("kl_divergence:", loss)
    print("ys", seen_ys)
    break
    
# print(y_train)
#     pipeline = Pipeline([
#         ('eeg_feautures', CustomTransformer()),
#         ('classifier', MLPClassifier(solver='lbfgs', activation='relu', random_state=42, 
#                     hidden_layer_sizes=(100, 50)))
#     ])

    # Train the model on the training data
#     lda = LinearDiscriminantAnalysis()
#     lda.fit(X_train, y_train)

#     pipeline.fit(X_train, y_train)

    # Evaluate the model on the testing data
#     score = pipeline.score(X_test, y_test)
#     print("cv score:", score)

    # Append the score to the list
#     cv_scores.append(score)
#     break

# 2415314690-0
# 3630961636-0 -- no nans

New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New New 

In [26]:
# TEST_PATH_EEG = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/'
test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
test = test.assign(eeg_sub_id='h') 
test = test.assign(eeg_label_offset_seconds=0) 
# test.loc[1] = test.loc[0]
print('Test shape',test.shape)
display(test.head())

test_feats = test.apply(lambda row: getAndExtractEEG(row, data_type="test"), axis=1)
test_feats = torch.FloatTensor(test_feats)
test_feats= test_feats.view(-1, input_dim)

with torch.no_grad():
    model.eval()
    preds = model(test_feats)
    print(preds)



Test shape (1, 5)


,spectrogram_id,eeg_id,patient_id,eeg_sub_id,eeg_label_offset_seconds
0,853520,3911565283,6885,h,0


New tensor([[6.1637e-01, 3.0420e-05, 3.9310e-05, 6.6391e-05, 3.5776e-01, 2.5741e-02]])


In [27]:
sub = pd.DataFrame({'eeg_id':test.eeg_id.values})
sub[TARGETS] = preds
sub.to_csv('submission.csv',index=False)
print('Submissionn shape',sub.shape)
sub.head()

Submissionn shape (1, 7)


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.616366,0.00003,0.000039,0.000066,0.357757,0.025741


In [28]:
torch.cuda.is_available():
# with open('my_dict.pkl', 'wb') as file:
#     pickle.dump(features_dict, file)
    
    

SyntaxError: invalid syntax (909394761.py, line 1)

In [ ]:
# pipeline = Pipeline([
#     ('eeg_feautures', CustomTransformer()),
#     ('classifier', LinearDiscriminantAnalysis())
# ])

# sub_data = data.iloc[:10]
# sub_labels = labels.iloc[:10]
# pipeline.fit(sub_data, sub_labels)

In [ ]:
# test_data = data.iloc[88:99]
# preds = pipeline.predict(test_data)

# print(preds)